## Homework #2
**Author**: Anna Vasylytsya

[Homework assignment prompt](https://github.com/DataTalksClub/mlops-zoomcamp/blob/main/02-experiment-tracking/homework.md)

In [1]:
import mlflow

## Question 1

In [2]:
! mlflow --version

mlflow, version 1.26.0


In [3]:
## av.note: alternative way of getting the version
# mlflow.__version__

## Question 2

In [4]:
## modified preprocess.py to pull directly from website using pandas.read_parquet(url)

In [5]:
!python preprocess.py --dest_path output

In [6]:
!dir "output" /b

dv.pkl
test.pkl
train.pkl
valid.pkl


**Answer**: There are four files in the output directory. 

## Question 3

In [7]:
!python train.py --data_path output

2022/05/30 03:00:08 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "C:\ProgramData\Anaconda3\envs\mlops\lib\site-packages\_distutils_hack\__init__.py:30: UserWarning: Setuptools is replacing distutils."


In [8]:
client = mlflow.tracking.MlflowClient(tracking_uri = 'sqlite:///mlflow.db')
experiment = client.get_experiment_by_name('nyc-taxi-experiment-hw2')
runs = client.search_runs(1)

**Answer**:

In [9]:
print('There are ', len(runs[0].data.params), ' parameters logged.')

There are  17  parameters logged.


## Question 4. Launch the tracking server locally

Ran this in command line: 

mlflow server --backend-store-uri "sqlite:///mlflow.db" --default-artifact-root "./artifacts"

**Answer**: `--default-artifact-root` is required. The other three choices are optional. [Source](https://www.mlflow.org/docs/latest/cli.html#mlflow-server)

## Question 5. Tune the hyperparameters of the model

Your task is to modify the script hpo.py and make sure that the validation RMSE is logged to MLflow for each run of the hyperparameter optimization (you will need to add a few lines of code to the objective function) and run the script without passing any parameters.

In [10]:
!python hpo.py


100%|##########| 50/50 [08:40<00:00, 10.41s/trial, best loss: 6.6284257482044735]


In [11]:
best_run = client.search_runs(
  experiment_ids = 2, 
  run_view_type= mlflow.entities.ViewType.ACTIVE_ONLY,
  max_results=1,
  order_by=["metrics.rmse ASC"]
)[0]
# best_run

**Answer**:

In [12]:
print('The best validation rmse is :', round(best_run.data.metrics['rmse'], 3))

The best validation rmse is : 6.628


av.note to self: the tracking server actually captures the script that the model was run from in the source column!

## Question 6. Promote the best model to the model registry

In [15]:
!python register_model.py

Registered model 'Best_RandomForestRegressor_BasedOn_202103_testdata' already exists. Creating a new version of this model...
2022/05/30 03:12:27 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: Best_RandomForestRegressor_BasedOn_202103_testdata, version 3
Created version '3' of model 'Best_RandomForestRegressor_BasedOn_202103_testdata'.


In [27]:
best_run_test = client.search_runs(
  experiment_ids = 3, 
  run_view_type= mlflow.entities.ViewType.ACTIVE_ONLY,
  max_results=1,
  order_by=["metrics.test_rmse ASC"]
)[0]

best_run_test.data.metrics['test_rmse']

6.547886185595423

In [36]:
# best_run_test.info.run_id

In [35]:
# model = client.get_registered_model('Best_RandomForestRegressor_BasedOn_202103_testdata')